<a href="https://colab.research.google.com/github/gracebrazil28/Tensorflow_Colab_Notebooks/blob/main/Regression_Decision_Tree_Melbourne_Housing_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Melbourne Housing Data from Kaggle: https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot

In [2]:
from google.colab import files
uploaded = files.upload()

Saving melb_data.csv to melb_data.csv


Import Pandas and Sklearn libraries


In [3]:
import pandas as pd
from sklearn.metrics import mean_absolute_error # For finding the best leaf node to split
from sklearn.model_selection import train_test_split # Splits the data set
from sklearn.tree import DecisionTreeRegressor # Build the whole decision tree regressor

In [3]:
# print(uploaded)  # checking if file got uploaded properly

Read uploaded csv file as home data, pick target variable and pick features to evaluate target on

In [4]:
import io
home_data = pd.read_csv(io.BytesIO(uploaded['melb_data.csv']))
home_data = home_data.dropna() #clean up

# Create target
y = home_data.Price
# Pick out features
features = ['BuildingArea', 'YearBuilt', 'Rooms', 'Landsize', 'Bathroom', 'Bedroom2', 'Car']
x = home_data[features]

Split training and testing data. Pick model and fit training data to model.

In [5]:
train_x, validate_x, train_y, validate_y = train_test_split(x, y, random_state=1)

DT_model = DecisionTreeRegressor(random_state=1)
DT_model.fit(train_x, train_y)

DecisionTreeRegressor(random_state=1)

Make Validation Predictions

In [6]:
validate_pred = DT_model.predict(validate_x)
validate_mae = mean_absolute_error(validate_pred, validate_y)
print("Validation Mean Absolute Error: "+ str(validate_mae))


Validation Mean Absolute Error: 361444.39703034214


Find the best leaf node with the least MAE in order to optimize leaf node

In [7]:
def calculate_mae(max_leaf_nodes, train_x, validate_x, train_y, validate_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_x, train_y)
    pred_y = model.predict(validate_x)
    mae = mean_absolute_error(validate_y, pred_y)
    return(mae)

candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]

# Write loop to find the ideal tree size from candidate_max_leaf_nodes
i=-1
least_mae=500000

for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae = calculate_mae(max_leaf_nodes, train_x, validate_x, train_y, validate_y)
    # print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))
    if least_mae > my_mae:
        least_mae = my_mae #lower mae is recorded
        i+=1 #increase the counter for change
    #print(i)


# Store the best value of max_leaf_nodes (it will be either 5, 25, 50, 100, 250 or 500)
best_tree_size = candidate_max_leaf_nodes[i]
print("Best Leaf Node: %d" %(best_tree_size))

Best Leaf Node: 100


Re-train Model and fit data

In [8]:
final_model = DecisionTreeRegressor(max_leaf_nodes = best_tree_size, random_state=1)
final_model.fit(x, y)

DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)